## Ashwani Singh

#### Importing all the require library

In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
import keras

import warnings
warnings.filterwarnings("ignore")

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

Using TensorFlow backend.


#### Path for the data set which is taken from UCI machine library.

In [5]:
train_path='https://archive.ics.uci.edu/ml/machine-learning-databases/poker/poker-hand-training-true.data'
test_path='https://archive.ics.uci.edu/ml/machine-learning-databases/poker/poker-hand-testing.data'

In [6]:
#Set the columns name for the data set

col_names=['s1','c1','s2','c2','s3','c3','s4','c4','s5','c5','class' ]

In [7]:
# Reading the data from path and assigning variables name

poker_train=pd.read_table(train_path, sep=',', names=col_names)
poker_test=pd.read_table(test_path, sep=',', names=col_names)

In [8]:
poker_train.head()
poker_test.head()

,s1,c1,s2,c2,s3,c3,s4,c4,s5,c5,class
0,1,10,1,11,1,13,1,12,1,1,9
1,2,11,2,13,2,10,2,12,2,1,9
2,3,12,3,11,3,13,3,10,3,1,9
3,4,10,4,11,4,1,4,13,4,12,9
4,4,1,4,13,4,12,4,11,4,10,9


,s1,c1,s2,c2,s3,c3,s4,c4,s5,c5,class
0,1,1,1,13,2,4,2,3,1,12,0
1,3,12,3,2,3,11,4,5,2,5,1
2,1,9,4,6,1,4,3,2,3,9,1
3,1,4,3,13,2,13,2,1,3,6,1
4,3,10,2,7,1,2,2,11,4,9,0


In [9]:
train=poker_train
test=poker_test

In [10]:
# Convert the suit of cards variables into category

col=['s1','s2','s3','s4','s5']
for i in col:
    train[i]=train[i].astype('category')
    test[i]=test[i].astype('category')

In [11]:
# One hot encoding for the categorical Variable

train=pd.get_dummies(train)
test=pd.get_dummies(test)

In [12]:
train.head()

,c1,c2,c3,c4,c5,class,s1_1,s1_2,s1_3,s1_4,...,s3_3,s3_4,s4_1,s4_2,s4_3,s4_4,s5_1,s5_2,s5_3,s5_4
0,10,11,13,12,1,9,1,0,0,0,...,0,0,1,0,0,0,1,0,0,0
1,11,13,10,12,1,9,0,1,0,0,...,0,0,0,1,0,0,0,1,0,0
2,12,11,13,10,1,9,0,0,1,0,...,1,0,0,0,1,0,0,0,1,0
3,10,11,1,13,12,9,0,0,0,1,...,0,1,0,0,0,1,0,0,0,1
4,1,13,12,11,10,9,0,0,0,1,...,0,1,0,0,0,1,0,0,0,1


Now we have 26 variables out of which 'class' is the target variable. So, the total number of explantory variables will be 25 and one target variable. Target variable has total no. 10 classes so, first we have to convert it into category and then get dummy variables for class.

In [220]:
x_train=np.array(train.drop('class', axis=1))
y=pd.get_dummies(train['class'].astype('category'))
y_train=np.array(y)

x_test=np.array(test.drop('class', axis=1))
y1=pd.get_dummies(test['class'].astype('category'))
y_test=np.array(y1)

## Tensorflow- Neural Net
Here I am using neural net for classification. For that I am concidering 3 hidden layers with 500 nodes at each layer.

In [240]:
n_nodes_hl1=500
n_nodes_hl2=500
n_nodes_hl3=500

n_classes=10

x=tf.placeholder('float',[None, 25])
y=tf.placeholder('float',[None,n_classes])

Activation function- For hidden layers i am cosidering tanh and for the output layer i am taking softmax. Here i am using Adam optimizer and with learning rate 0.2.

In [246]:
def neural_network_model(data):
    hidden_1_layer={'weights':tf.Variable(tf.random_normal([25, n_nodes_hl1])),
                   'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}
    
    hidden_2_layer={'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                   'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))}
    
    hidden_3_layer={'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                   'biases':tf.Variable(tf.random_normal([n_nodes_hl3]))}

    output_layer={'weights':tf.Variable(tf.random_normal([n_nodes_hl3,n_classes])),
                   'biases':tf.Variable(tf.random_normal([n_classes]))}

    l1=tf.add(tf.matmul(data,hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1=tf.nn.tanh(l1)
    
    l2=tf.add(tf.matmul(l1,hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2=tf.nn.tanh(l2)

    l3=tf.add(tf.matmul(l1,hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3=tf.nn.tanh(l3)
    
    output=tf.matmul(l3,output_layer['weights'])+ output_layer['biases']
    output=tf.nn.softmax(output)
    return output

Simple Model without batch processesing.

In [247]:
def train_neural_network(x):
    prediction=neural_network_model(x)
    cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
    
    learning_rate= 0.1
    optimizer= tf.train.AdamOptimizer(learning_rate).minimize(cost)
    hm_epochs=10
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(hm_epochs):
            epoch_loss=0
            _,c=sess.run([optimizer, cost], feed_dict={x: x_train, y: y_train})
            epoch_loss+=c
            print('Epoch', epoch, 'completed out of', hm_epochs,'loss: ',epoch_loss)
        correct=tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))
        accuracy= tf.reduce_mean (tf.cast(correct, 'float'))
        print('Accuracy:', accuracy.eval({x:x_test, y:y_test}))

In [248]:
train_neural_network(x)

Epoch 0 completed out of 10 loss:  2.43750786781
Epoch 1 completed out of 10 loss:  2.01901054382
Epoch 2 completed out of 10 loss:  1.96142196655
Epoch 3 completed out of 10 loss:  1.96149492264
Epoch 4 completed out of 10 loss:  1.96149492264
Epoch 5 completed out of 10 loss:  1.96149492264
Epoch 6 completed out of 10 loss:  1.96149492264
Epoch 7 completed out of 10 loss:  1.96149492264
Epoch 8 completed out of 10 loss:  1.96149492264
Epoch 9 completed out of 10 loss:  1.96149492264
Accuracy: 0.501209


## Result
The accuracy for the test data is- 50.12%

## Keras - Tensorflow
Importing models and layers from Keras Library 

In [198]:
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

Target variable has total no. 10 classes so, first we have to convert it into category and then get dummy variables for class.

In [202]:
x_train=np.array(train.drop('class', axis=1))
y=pd.get_dummies(train['class'].astype('category'))
y_train=np.array(y)

x_test=np.array(test.drop('class', axis=1))
y1=pd.get_dummies(test['class'].astype('category'))
y_test=np.array(y1)

Here I am using 'RELU' activation function for the hidden layers and 'SOFTMAX' activation
function for output Layers. For the optimization i used Adam Optimizer. At each hidden layer i took 500 nodes and at output 10 nodes because we have 10 classes. Here i am using 50 epoch and batch size 100.

In [199]:
n_input_node= 25
n_nodes_hl1=500
n_nodes_hl2=500
n_nodes_hl3=500
n_output_node = 10

epochs =50
batch_size = 100

model = Sequential([
 Dense(output_dim=n_nodes_hl1, input_dim=n_input_node, activation='relu'),
 Dense(output_dim=n_nodes_hl3, input_dim=n_nodes_hl1, activation='relu'),
 Dense(output_dim=n_nodes_hl3, input_dim=n_nodes_hl2, activation='relu'),
    
 Dense(output_dim=n_output_node, input_dim=n_nodes_hl3, activation='softmax'),
 ])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
trained_model_5d = model.fit(x_train, y_train, nb_epoch=epochs, batch_size=batch_size, 
                             validation_data=(x_test, y_test))

Train on 25010 samples, validate on 1000000 samples
Epoch 1/50
25010/25010 [==============================] - 76s - loss: 0.9941 - acc: 0.5121 - val_loss: 0.9650 - val_acc: 0.5323
Epoch 2/50
25010/25010 [==============================] - 68s - loss: 0.9565 - acc: 0.5408 - val_loss: 0.9491 - val_acc: 0.5469
Epoch 3/50
25010/25010 [==============================] - 69s - loss: 0.9480 - acc: 0.5468 - val_loss: 1.0082 - val_acc: 0.5267
Epoch 4/50
25010/25010 [==============================] - 70s - loss: 0.9355 - acc: 0.5587 - val_loss: 0.9427 - val_acc: 0.5540
Epoch 5/50
25010/25010 [==============================] - 68s - loss: 0.9276 - acc: 0.5638 - val_loss: 0.9304 - val_acc: 0.5614
Epoch 6/50
25010/25010 [==============================] - 68s - loss: 0.9152 - acc: 0.5753 - val_loss: 0.9192 - val_acc: 0.5703
Epoch 7/50
25010/25010 [==============================] - 68s - loss: 0.8975 - acc: 0.5852 - val_loss: 0.9068 - val_acc: 0.5812
Epoch 8/50
25010/25010 [============================

## Result
 i am getting max accuracy of test data at 48th epoch - 97.79%